## TESTE

In [ ]:
import sqlite3
import os

# Caminho corrigido com raw string (r"...")
db_path = r""

# Verifica se o caminho é válido
if os.path.exists(db_path):
    # Conectar ao banco SQLite
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Executar a consulta para obter os nomes das tabelas
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tabelas = cursor.fetchall()

    # Exibir os nomes das tabelas
    print("Tabelas no banco de dados:")
    for tabela in tabelas:
        print(tabela[0])

    # Fechar a conexão
    cursor.close()
    conn.close()
else:
    print("Arquivo não encontrado:", db_path)


Tabelas no banco de dados:
zsd173
Info_Gerais
Transp
Base_Clientes
Base_Produtos
Book
valor_dedicado
BI_Estadia
BI_Dedicado
TABELAS_FRACIONADO
TABELAS_LOTACAO
BI_Descarga
BI_Armazenagem
BI_Paletizacao
 BI_Paletizacao


In [ ]:
import sqlite3

# Conectar ao banco SQLite
conn = sqlite3.connect("G:")
cursor = conn.cursor()

# Executar a consulta para remover todos os dados da tabela Relatorio_Txt
cursor.execute("DELETE FROM Relatorio_Txt")
conn.commit()  # Confirmar a transação

# Fechar a conexão
cursor.close()
conn.close()

print("Todos os dados da tabela Relatorio_Txt foram removidos.")

Todos os dados da tabela Relatorio_Txt foram removidos.


In [ ]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados
conn = sqlite3.connect(r"G:")

cursor = conn.cursor()

# Consultar os dados da tabela 'valor_dedicado'
cursor.execute("SELECT * FROM valor_dedicado")

# Carregar os dados no pandas DataFrame
valor_dedicado_df = pd.DataFrame(cursor.fetchall(), columns=["Tipo veiculo", "min", "max", "valor"])

# Verificar os tipos de dados das colunas 'min' e 'max'
print(valor_dedicado_df[["min", "max"]].dtypes)

# Fechar a conexão
conn.close()


min    float64
max    float64
dtype: object


In [ ]:
import sqlite3

# Caminho do banco corrigido
db_path = r"G:"

# Conectar ao banco de dados
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Excluir a tabela
cursor.execute("DROP TABLE IF EXISTS BI_Dedicado")
conn.commit()

cursor.close()
conn.close()

print("Tabela excluída com sucesso.")



Tabela excluída com sucesso.


In [ ]:
import sqlite3

# Caminho do banco corrigido
db_path = r"G"

# Conectar ao banco de dados
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Excluir a tabela
cursor.execute("DROP TABLE IF EXISTS BI_Descarga_Excel")
cursor.execute("DROP TABLE IF EXISTS BI_Descarga_TXT")
conn.commit()

cursor.close()
conn.close()

print("Tabela excluída com sucesso.")

Tabela excluída com sucesso.


# Apagar e Criar Tabelas

In [ ]:
#Apagar
import sqlite3

conn = sqlite3.connect(r"G:")
cursor = conn.cursor()
# cursor.execute("DELETE FROM BI_Estadia")
# cursor.execute("DELETE FROM BI_Dedicado")
# cursor.execute("DELETE FROM BI_Descarga")
# cursor.execute("DELETE FROM BI_Armazenagem")
cursor.execute("DELETE FROM BI_Paletizacao")

conn.commit() 

# Fechar a conexão
cursor.close()
conn.close() 


In [ ]:
import sqlite3

conn = sqlite3.connect(r"G:")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS ' BI_Paletizacao'")

conn.commit() 

# Fechar a conexão
cursor.close()
conn.close() 


In [ ]:
import sqlite3
import pandas as pd
import os

caminho_banco = r"G"
caminho_excel = r"G:"

if not os.path.exists(caminho_excel):
    print(f"Erro: O arquivo '{caminho_excel}' não foi encontrado.")
else:
    try:
        conn = sqlite3.connect(caminho_banco)
        cursor = conn.cursor()

        df = pd.read_excel(caminho_excel)

        if df.empty:
            print("O arquivo Excel está vazio. Nenhum dado foi inserido.")
        else:
            # Inserir os dados na tabela (sem apagar o que já existe)
            df.to_sql("BI_Paletizacao", conn, if_exists="append", index=False)
            print("✅ Dados do Excel inseridos com sucesso na tabela 'BI_Paletizacao'.")

        conn.close()

    except Exception as e:
        print(f"❌ Erro ao processar os dados: {e}")


✅ Dados do Excel inseridos com sucesso na tabela 'BI_Paletizacao'.


In [ ]:
import sqlite3
import pandas as pd
import os

# Caminhos dos arquivos
caminho_banco = r"G"
caminho_excel = r"G"

# Verifica se o arquivo Excel existe
if not os.path.exists(caminho_excel):
    print(f"Erro: O arquivo '{caminho_excel}' não foi encontrado.")
else:
    try:
        # Conectar ao banco SQLite
        conn = sqlite3.connect(caminho_banco)
        cursor = conn.cursor()

        # Lendo os dados do Excel
        df = pd.read_excel(caminho_excel, engine="pyxlsb")

        if df.empty:
            print("O arquivo Excel está vazio. Nenhum dado foi inserido.")
        else:
            # Remover colunas completamente vazias
            df = df.dropna(axis=1, how="all")
            
            # Garantir que a coluna "Data Requisição" seja convertida corretamente
            if "Data Requisição" in df.columns:
                df["Data Requisição"] = pd.to_numeric(df["Data Requisição"], errors="coerce")
                df["Data Requisição"] = pd.to_datetime("1899-12-30") + pd.to_timedelta(df["Data Requisição"], unit="D")
                df["Data Requisição"] = df["Data Requisição"].dt.strftime("%d/%m/%Y")  # Formato DD/MM/YYYY
            
            # Criar a tabela se não existir
            colunas = ", ".join([f'"{col}" TEXT' for col in df.columns])
            cursor.execute(f"CREATE TABLE IF NOT EXISTS BI_Dedicado ({colunas})")

            # Apagar dados antigos antes de inserir novos
            cursor.execute("DELETE FROM BI_Dedicado")
            conn.commit()

            # Inserir os dados na tabela
            df.to_sql("BI_Dedicado", conn, if_exists="append", index=False)

            print("✅ Dados do arquivo Excel foram inseridos na tabela BI_Dedicado com sucesso!")

        # Fechar conexão
        conn.close()
    
    except Exception as e:
        print(f"❌ Erro ao processar os dados: {e}")

✅ Dados do arquivo Excel foram inseridos na tabela BI_Dedicado com sucesso!


In [ ]:
import sqlite3
import pandas as pd
import os

# Caminhos dos arquivos
caminho_banco = r"G"
caminho_excel = r"G"

# Verifica se o arquivo Excel existe
if not os.path.exists(caminho_excel):
    print(f"Erro: O arquivo '{caminho_excel}' não foi encontrado.")
else:
    try:
        # Conectar ao banco SQLite
        conn = sqlite3.connect(caminho_banco)
        cursor = conn.cursor()

        # Lendo os dados do Excel
        df = pd.read_excel(caminho_excel, engine="pyxlsb")

        if df.empty:
            print("O arquivo Excel está vazio. Nenhum dado foi inserido.")
        else:
            # Remover colunas completamente vazias
            df = df.dropna(axis=1, how="all")
            
            # Garantir que a coluna "Data Requisição" seja convertida corretamente
            if "Data Requisição" in df.columns:
                df["Data Requisição"] = pd.to_numeric(df["Data Requisição"], errors="coerce")
                df["Data Requisição"] = pd.to_datetime("1899-12-30") + pd.to_timedelta(df["Data Requisição"], unit="D")
                df["Data Requisição"] = df["Data Requisição"].dt.strftime("%d/%m/%Y")  # Formato DD/MM/YYYY
            
            # Criar a tabela se não existir
            colunas = ", ".join([f'"{col}" TEXT' for col in df.columns])
            cursor.execute(f"CREATE TABLE IF NOT EXISTS BI_Estadia ({colunas})")

            # Apagar dados antigos antes de inserir novos
            cursor.execute("DELETE FROM BI_Estadia")
            conn.commit()

                    # Converter todos os dados para string para evitar problemas de tipo
            df = df.astype(str)

            # Inserir os dados na tabela
            df.to_sql("BI_Estadia", conn, if_exists="append", index=False, dtype={col: 'TEXT' for col in df.columns})

            print("✅ Dados do arquivo Excel foram inseridos na tabela BI_Estadia com sucesso!")


        # Fechar conexão
        conn.close()
    
    except Exception as e:
        print(f"❌ Erro ao processar os dados: {e}")




✅ Dados do arquivo Excel foram inseridos na tabela BI_Estadia com sucesso!


In [ ]:
import sqlite3
import pandas as pd
import os

caminho_banco = r"G"
caminho_excel = r"G"

if not os.path.exists(caminho_excel):
    print(f"Erro: O arquivo '{caminho_excel}' não foi encontrado.")
else:
    try:
        conn = sqlite3.connect(caminho_banco)

        df = pd.read_excel(caminho_excel)

        if df.empty:
            print("O arquivo Excel está vazio. Nenhum dado foi inserido.")
        else:
            df = df.dropna(axis=1, how="all")
            df = df.loc[:, ~df.columns.str.contains("^Unnamed")]  # 🔥 Remove colunas vazias

        if "Data de Requisição" in df.columns:
            try:
                df["Data de Requisição"] = pd.to_datetime(df["Data de Requisição"], errors="coerce", dayfirst=True)
            except:
                df["Data de Requisição"] = pd.to_numeric(df["Data de Requisição"], errors="coerce")
                df.loc[(df["Data de Requisição"] > 60000) | (df["Data de Requisição"] < 0), "Data de Requisição"] = None
                df["Data de Requisição"] = pd.to_datetime("1899-12-30") + pd.to_timedelta(df["Data de Requisição"], unit="D")

            df["Data de Requisição"] = df["Data de Requisição"].dt.strftime("%d/%m/%Y")

        df = df.astype(str)

        df.to_sql("BI_Descarga", conn, if_exists="append", index=False, dtype={col: 'TEXT' for col in df.columns})

        print("✅ Dados do arquivo Excel foram inseridos na tabela BI_Descarga com sucesso!")

        conn.close()

    except Exception as e:
        print(f"❌ Erro ao processar os dados do Excel: {e}")


✅ Dados do arquivo Excel foram inseridos na tabela BI_Descarga com sucesso!


In [ ]:
import sqlite3
import pandas as pd
import os

caminho_banco = r"G"
caminho_excel = r"G"
if not os.path.exists(caminho_excel):
    print(f"Erro: O arquivo '{caminho_excel}' não foi encontrado.")
else:
    try:
        conn = sqlite3.connect(caminho_banco)
        cursor = conn.cursor()

        df = pd.read_excel(caminho_excel)

        if df.empty:
            print("O arquivo Excel está vazio. Nenhum dado foi inserido.")
        else:
            df = df.dropna(axis=1, how="all")

        if "Data Requisição" in df.columns:
            # Tenta converter diretamente para datetime (se já estiver em string)
            try:
                df["Data Requisição"] = pd.to_datetime(df["Data Requisição"], errors="coerce", dayfirst=True)
            except:
                # Se falhar, tenta tratar como número serial do Excel
                df["Data Requisição"] = pd.to_numeric(df["Data Requisição"], errors="coerce")
                df.loc[(df["Data Requisição"] > 60000) | (df["Data Requisição"] < 0), "Data Requisição"] = None
                df["Data Requisição"] = pd.to_datetime("1899-12-30") + pd.to_timedelta(df["Data Requisição"], unit="D")

            # Formata para string padrão
            df["Data Requisição"] = df["Data Requisição"].dt.strftime("%d/%m/%Y")


            colunas = ", ".join([f'"{col}" TEXT' for col in df.columns])
            cursor.execute(f"CREATE TABLE IF NOT EXISTS BI_Armazenagem ({colunas})")

            cursor.execute("DELETE FROM BI_Armazenagem")
            conn.commit()

            df = df.astype(str)
            df.to_sql("BI_Armazenagem", conn, if_exists="append", index=False, dtype={col: 'TEXT' for col in df.columns})

            print("✅ Dados do arquivo Excel foram inseridos na tabela BI_Armazenagem com sucesso!")

        conn.close()

    except Exception as e:
        print(f"❌ Erro ao processar os dados: {e}")


✅ Dados do arquivo Excel foram inseridos na tabela BI_Armazenagem com sucesso!


In [ ]:
import sqlite3

caminho_banco = r"G"

try:
    conn = sqlite3.connect(caminho_banco)
    cursor = conn.cursor()

    # Comando para renomear a tabela
    cursor.execute("ALTER TABLE Relatorio_Txt RENAME TO BI_Paletizacao")
    conn.commit()
    print("✅ Tabela renomeada com sucesso!")

    conn.close()

except Exception as e:
    print(f"❌ Erro ao renomear a tabela: {e}")


✅ Tabela renomeada com sucesso!


In [ ]:
import pandas as pd

# Caminho do arquivo Excel
caminho_excel = r"C:"

# Caminho da pasta de saída dos arquivos TXT
caminho_saida = r"G"

# Nomes das abas
abas = ["724", "725"]

for aba in abas:
    # Leitura da aba específica
    df = pd.read_excel(caminho_excel, sheet_name=aba)
    
    # Defina aqui as colunas desejadas ou use todas as colunas
    colunas_desejadas = df.columns  # ou por exemplo: ['coluna1', 'coluna2', ...]
    
    # Caminho completo para salvar o txt
    caminho_txt_proc = fr"{caminho_saida}\{aba}.txt"
    
    # Exportação para TXT
    df[colunas_desejadas].to_csv(caminho_txt_proc, sep="|", index=False, encoding="utf-8")
    
    print(f"Arquivo {aba}.txt gerado com sucesso em {caminho_saida}")
